In [159]:
from langchain.tools import OpenAPISpec, APIOperation
from langchain.chains import OpenAPIEndpointChain
from langchain.requests import Requests
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

In [160]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
llm = OpenAI(temperature=0)

In [151]:
openapi_chain = get_openapi_chain(
    "https://www.klarna.com/us/shopping/public/openai/v0/api-docs/"
)

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [152]:
question = "What is the most recommended shoe brand?"

In [153]:
query_response = openapi_chain.run(question)

In [154]:
print(query_response)

{'products': [{'name': 'Jordan Retro Mens Shoes', 'url': 'https://www.klarna.com/us/shopping/pl/cl337/3207231308/Shoes/Jordan-Retro-Mens-Shoes/?utm_source=openai&ref-site=openai_plugin', 'price': '$209.99', 'attributes': ['Outsole:Rubber', 'Fastening:Laced', 'Color:White,Yellow,Black', 'Upper Material:Leather', 'Size (US):11,9.5,12,15,11.5,10.5,7,8,9,7.5,8.5,12.5,10', 'Series:Nike Air Jordan,Nike Air Jordan 1']}, {'name': 'Nike Dunk Low W - White/Black', 'url': 'https://www.klarna.com/us/shopping/pl/cl337/3200134705/Shoes/Nike-Dunk-Low-W-White-Black/?utm_source=openai&ref-site=openai_plugin', 'price': '$110.00', 'attributes': ['Outsole:Rubber', 'Fastening:Laced', 'Midsole:Foam', 'Target Group:Woman', 'Color:Black', 'Upper Material:Leather', 'Size (US):9.5,10,11,12,13,14,15,16,17,18,11.5,10.5,2,3,4,5,6,7,8,9,2.5,3.5,4.5,16.5,5.5,15.5,6.5,14.5,13.5,7.5,8.5,12.5', 'Series:Nike Dunk']}, {'name': 'Nike Dunk Low Panda M - Black/White', 'url': 'https://www.klarna.com/us/shopping/pl/cl337/3200

In [155]:
template = "Translate the following API response into a concise, consistent, and human-readable sentence, that answer a customer's question about a Klarna product or products.  The product name, its price, and the link to view or purchase it online should be available to the user."

In [156]:
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [157]:
human_template = "{question}-----{json_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt
)

In [158]:
chain.run(question=question, json_text=str(query_response))

"The most recommended shoe brand is Nike. Some popular Nike shoes include the Jordan Retro Mens Shoes priced at $209.99, the Nike Dunk Low W - White/Black priced at $110.00, the Nike Dunk Low Panda M - Black/White priced at $124.00, and the Nike Air Force 1 '07 M - White priced at $76.50. You can view or purchase these shoes online using the provided links."